# feats using polar yay

In [19]:
import polars as pl
import yaml
from pathlib import Path

# pl.set_option("display", False)

In [20]:
input_path = Path("../data/cleaned/2015_2024_statcast_clean.parquet")

df = pl.read_parquet(input_path)

In [21]:
df = df.sort(
    [
        "game_date",
        "game_pk",
        "at_bat_number",
        "pitch_number",
    ],
    descending=[False, False, False, True],
)


df.head()
df.tail()


pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,zone,stand,p_throws,type,hit_location,bb_type,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,bat_speed,swing_length
str,datetime[μs],f64,f64,f64,str,i64,i64,str,f64,str,str,str,f64,str,i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,str,str,f64,f64,f64
"""ST""",2024-10-20 00:00:00,78.8,4.1,5.27,"""Manaea, Sean""",571970,640455,"""walk""",14.0,"""L""","""L""","""B""",null,null,3,1,-1.04,0.07,1.44,1.65,null,null,null,0,3,"""Bot""",null,null,-3.876776,-114.790473,-1.213129,-8.519711,20.865706,-31.531093,2.99,1.5,null,null,null,79.7,2141.0,6.8,775302,53.72,null,0.689131,0.7,1.0,0.0,0.0,null,31,5,4,1,"""Infield shade""","""Standard""",294.0,null,null
"""CH""",2024-10-20 00:00:00,83.9,4.16,5.13,"""Manaea, Sean""",571970,640455,null,13.0,"""L""","""L""","""B""",null,null,2,1,1.28,-0.01,-1.13,1.32,null,null,null,0,3,"""Bot""",null,null,-14.910726,-121.300964,-2.306161,15.532788,23.944392,-32.030777,2.99,1.4,null,null,null,84.7,1835.0,7.2,775302,53.31,null,null,null,null,null,null,null,31,4,4,1,"""Infield shade""","""Standard""",106.0,null,null
"""SI""",2024-10-20 00:00:00,91.9,4.03,4.97,"""Manaea, Sean""",571970,640455,null,13.0,"""L""","""L""","""B""",null,null,1,1,1.16,0.76,-0.93,2.17,null,null,null,0,3,"""Bot""",null,null,-15.210428,-132.944842,-2.837582,16.955867,27.981449,-22.734943,2.99,1.53,null,null,null,92.7,2131.0,7.0,775302,53.48,null,null,null,null,null,null,null,31,3,4,1,"""Infield shade""","""Standard""",113.0,null,null
"""ST""",2024-10-20 00:00:00,80.2,3.84,5.31,"""Manaea, Sean""",571970,640455,null,13.0,"""L""","""L""","""B""",null,null,0,1,-1.34,0.15,-0.86,1.28,null,null,null,0,3,"""Bot""",null,null,-7.987162,-116.480556,-2.438514,-10.452965,23.959752,-30.532682,2.99,1.35,null,null,null,81.3,2117.0,7.4,775302,53.14,null,null,null,null,null,null,null,31,2,4,1,"""Infield shade""","""Standard""",284.0,null,null
"""ST""",2024-10-20 00:00:00,78.9,3.89,5.26,"""Manaea, Sean""",571970,640455,null,8.0,"""L""","""L""","""S""",null,null,0,0,-1.57,0.19,-0.23,1.53,null,null,null,0,3,"""Bot""",null,null,-6.125394,-114.782271,-1.669025,-12.671426,21.628193,-30.380521,2.99,1.48,null,null,null,79.9,2152.0,7.0,775302,53.45,null,null,null,null,null,null,null,31,1,4,1,"""Infield shade""","""Standard""",283.0,null,null


In [ ]:
# print how many pitch_types are null

print(len(df.filter(pl.col("pitch_type").is_null())))

df = df.drop_nulls(subset=["pitch_type", "pitcher", "batter"])
print(len(df.filter(pl.col("pitch_type").is_null())))

0
0
